# 1 写csv

In [ ]:
import os
from PIL import Image
import torchvision
from torchvision import transforms
import csv
import time
from torch.autograd import Variable
from torchvision import models
import torch
print('start')
start=time.time()
infile="/home/user01/envoriments/data/testb/"
csvfile_root='/home/user01/envoriments/question.csv'
image_id='Images/'

small_class_num={ 'coat':8, 'collar':5, 'lapel':5, 'neck':5, 'neckline':10, 'pant':6, 'sleeve':9, 'skirt':6}
retrain_model_class={ 'coat':'densenet_fashionai_coat.pkl',
                     'collar':'densenet_fashionai_collar_en.pkl',
                     'lapel':'densenet_fashionai_lapel.pkl',
                     'neck':'densenet_fashionai_neck_en.pkl', 
                     'neckline':'densenet_fashionai_neckline_en.pkl', 
                     'pant':'densenet_fashionai_pant.pkl',
                     'sleeve':'densenet_fashionai_sleeve_en.pkl',
                     'skirt':'densenet_fashionai_skirt.pkl'}

def get_strfeature(model,img_tensor):
    model.train(False)
    img_tensor=Variable(img_tensor)
    outputs=model(img_tensor)
    outputs=torch.nn.functional.softmax(outputs)
    n=outputs.data.numpy()
    x=n[0].tolist()
    s=(str(round(i, 4)) for i in x)
    s=';'.join(s)
    return s

model_reload=models.densenet169()

def load_model(phase):#加载预训练的模型
    out_features=small_class_num[phase]
    model_reload.classifier=torch.nn.Linear(1664,out_features)    
    url=retrain_model_class[phase]
    model_reload.load_state_dict(torch.load(url))
    model_reload.eval()
    return model_reload

def image_transfroms():# 把数据转换成模型指定大小，并转换成tensor类型
    return transforms.Compose([
        transforms.Scale(224),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
rows=[]
with open(csvfile_root,'r') as f:
        #image_id=image_id+image
        csv_file=csv.reader(f)
        rowss=[row for row in csv_file]
        rows=rowss

def get_result():#从文件夹里读取每一张图片并送入训练好的模型
    big_classes=os.listdir(infile)
    s=1
    for big_class in big_classes:
        print(s)
        s+=1
        images=os.listdir(infile+big_class)
        i=big_class.find('_')
        class_name=big_class[:i]
        model=load_model(class_name)
        for image in images:
            img=Image.open(infile+big_class+'/'+image)
            img_tensor=image_transfroms()(img)
            img_tensor.resize_(1,3,224,224)
            strfeature=get_strfeature(model,img_tensor)
            image_id2=image_id+big_class+'/'+image
            for i,row in enumerate(rows):
                if row[0]==image_id2:
                    rows[i][2]=strfeature
                    break          
    return rows                    
rows=get_result()
time_elapsed=time.time()-start
print('读出和替换花费时间为{:.4f}分：{:.4f}秒'.format(time_elapsed//60,time_elapsed%60))
print('开始整体写入')
with open(csvfile_root,"w",newline='') as csvfile:
                        writer = csv.writer(csvfile)
                        for row in rows:
                            writer.writerow(row)
time_elapsed1=time.time()-start
print('end')
print('共花费时间为{:.4f}分：{:.4f}秒'.format(time_elapsed1//60,time_elapsed1%60))

# 2 融合csv

In [1]:
import os
from PIL import Image
import torchvision
from torchvision import transforms
import csv
import time
csvfile_root1='/home/user01/envoriments/answer_boost.csv'
csvfile_root2='/home/user01/envoriments/question_add6.csv'
csvfile_root3='/home/user01/envoriments/answerb1.csv'
with open(csvfile_root1,'r') as f1:
    #image_id=image_id+image
    csv_file1=csv.reader(f1)
    rows1=[row for row in csv_file1]
with open(csvfile_root2,'r') as f2:
    #image_id=image_id+image
    csv_file2=csv.reader(f2)
    rows2=[row for row in csv_file2]
with open(csvfile_root3,'r') as f3:
    #image_id=image_id+image
    csv_file3=csv.reader(f3)
    rows3=[row for row in csv_file3]         
finalrow=rows1#使用切片赋值法指的不是同一个地址
print('end')

end


In [5]:
csvfile_root4='/home/user01/envoriments/questionfinal.csv'
for i in range(15042):
    strarray1=rows1[i][2].split(";")
    strarray2=rows2[i][2].split(";")
    strarray3=rows3[i][2].split(";")
    
    floatarray1=[0 for x in range(len(strarray2))]
    floatarray2=[0 for x in range(len(strarray2))]
    floatarray3=[0 for x in range(len(strarray2))]
    for j in range(len(strarray2)):
        floatarray1[j]=round(float(strarray1[j]),4)
        floatarray2[j]=round(float(strarray2[j]),4)
        floatarray3[j]=round(float(strarray3[j]),4)
    
    max1=max(floatarray1)
    max2=max(floatarray2)
    max3=max(floatarray3)
    index1=floatarray1.index(max1)
    index2=floatarray2.index(max2)
    index3=floatarray3.index(max3)
    if index1==index2==index3:
        maxmax=max([max1,max2,max3])
        finalrow[i]=rows1[i]
    else :
        if index1==index2:
            if max1>=max2:
                finalrow[i]=rows1[i]
            else :
                finalrow[i]=rows2[i]
        elif index1==index3:
            if max1>=max3:
                finalrow[i]=rows1[i]
            else :
                finalrow[i]=rows3[i]
        elif index2==index3:
            if max2>=max3:
                finalrow[i]=rows2[i]
            else :
                finalrow[i]=rows3[i]
        else :
            floatarray4=[floatarray1[i]+floatarray2[i]+floatarray3[i] for i in range(len(strarray2))]
            
            index4=floatarray4.index(max(floatarray4))
            if index4==index1:
                finalrow[i]=rows1[i]
            elif index4==index2:
                finalrow[i]=rows2[i]
            elif index4==index3:
                finalrow[i]=rows3[i]
            else :
                listrow=[round(floatarray4[i]/3,4) for i in range(len(strarray2))]
                strow=(str(i) for i in listrow)
                strow1=';'.join(strow)
                finalrow[i][2]=strow1
with open(csvfile_root4,"w",newline='') as csvfile:
                writer = csv.writer(csvfile)
                for row in finalrow:
                    writer.writerow(row)           
            
print('end')



end


# 3 找出三个模型结果不同的图片，并启动到一个新的文件夹

In [9]:
import os
from PIL import Image
import torchvision
from torchvision import transforms
import csv
import time
csvfile_root1='/home/user01/envoriments/question.csv'
csvfile_root2='/home/user01/envoriments/answerb1.csv'
csvfile_root3='/home/user01/envoriments/answer6.csv'
with open(csvfile_root1,'r') as f1:
    #image_id=image_id+image
    csv_file1=csv.reader(f1)
    rows1=[row for row in csv_file1]
with open(csvfile_root2,'r') as f2:
    #image_id=image_id+image
    csv_file2=csv.reader(f2)
    rows2=[row for row in csv_file2]
with open(csvfile_root3,'r') as f3:
    #image_id=image_id+image
    csv_file3=csv.reader(f3)
    rows3=[row for row in csv_file3]         
finalrow=rows1#使用切片赋值法指的不是同一个地址
print('end')

end


In [19]:
import os
from PIL import Image
import torchvision
from torchvision import transforms
import csv
import time
from torch.autograd import Variable
from torchvision import models
import torch
import shutil
infile="/home/user01/envoriments/data/testb"
outfile='/home/user01/envoriments/data/testb_error'
q=0
'''if not os.path.exists(os.path.join(outfile,"coat_length_labels")):
    os.makedirs(os.path.join(outfile,"coat_length_labels"))
if not os.path.exists(os.path.join(outfile,"collar_design_labels")):
    os.makedirs(os.path.join(outfile,"collar_design_labels"))    
if not os.path.exists(os.path.join(outfile,"lapel_design_labels")):
    os.makedirs(os.path.join(outfile,"lapel_design_labels"))    
if not os.path.exists(os.path.join(outfile,"neck_design_labels")):
    os.makedirs(os.path.join(outfile,"neck_design_labels"))
if not os.path.exists(os.path.join(outfile,"neckline_design_labels")):
    os.makedirs(os.path.join(outfile,"neckline_design_labels"))
if not os.path.exists(os.path.join(outfile,"pant_length_labels")):
    os.makedirs(os.path.join(outfile,"pant_length_labels"))
if not os.path.exists(os.path.join(outfile,"sleeve_length_labels")):
    os.makedirs(os.path.join(outfile,"sleeve_length_labels"))
if not os.path.exists(os.path.join(outfile,"skirt_length_labels")):
    os.makedirs(os.path.join(outfile,"skirt_length_labels"))'''    
for i in range(15042):
    strarray1=rows1[i][2].split(";")
    strarray2=rows2[i][2].split(";")
    strarray3=rows3[i][2].split(";")
    
    floatarray1=[0 for x in range(len(strarray1))]
    floatarray2=[0 for x in range(len(strarray2))]
    floatarray3=[0 for x in range(len(strarray3))]
    for j in range(len(strarray1)):
        floatarray1[j]=round(float(strarray1[j]),4)
        floatarray2[j]=round(float(strarray2[j]),4)
        floatarray3[j]=round(float(strarray3[j]),4)
    max1=max(floatarray1)
    max2=max(floatarray2)
    max3=max(floatarray3)
    index1=floatarray1.index(max1)
    index2=floatarray2.index(max2)
    index3=floatarray3.index(max3)
    if not index1==index2==index3:
        q+=1
        oldname=os.path.join(infile,rows1[i][1],rows1[i][0].split("/")[2])
        newname=os.path.join(outfile,rows1[i][1],rows1[i][0].split("/")[2])
        shutil.copy(oldname,newname)
print("end",q)            
            
        



end 3202


# 4 用残差模型，来对之前找到的误差图片，重新获取特征，并且在csv文件里替换

In [2]:
import os
from PIL import Image
import torchvision
from torchvision import transforms
import csv
import time
from torch.autograd import Variable
from torchvision import models
import torch
print('start')
start=time.time()
infile="/home/user01/envoriments/data/testb_error/"
csvfile_root='/home/user01/envoriments/question_add6.csv'
image_id='Images/'
small_class_num={ 'coat':8, 'collar':5, 'lapel':5, 'neck':5, 'neckline':10, 'pant':6, 'sleeve':9, 'skirt':6}
retrain_model_class={ 'coat':'densenet121_fashionai_coat.pkl',
                     'collar':'densenet121_fashionai_collar.pkl',
                     'lapel':'densenet121_fashionai_lapel.pkl',
                     'neck':'densenet121_fashionai_neck.pkl', 
                     'neckline':'densenet121_fashionai_neckline.pkl', 
                     'pant':'densenet121_fashionai_pant.pkl',
                     'sleeve':'densenet121_fashionai_sleeve.pkl',
                     'skirt':'densenet121_fashionai_skirt.pkl'}

def get_strfeature(model,img_tensor):
    img_tensor=Variable(img_tensor)
    outputs=model(img_tensor)
    outputs=torch.nn.functional.softmax(outputs)
    n=outputs.data.numpy()
    x=n[0].tolist()
    s=(str(round(i, 4)) for i in x)
    s=';'.join(s)
    return s

model_reload=models.densenet121()



def load_model(phase):#加载预训练的模型
    out_features=small_class_num[phase]
    model_reload.classifier=torch.nn.Linear(1024,out_features)
    url=retrain_model_class[phase]
    model_reload.load_state_dict(torch.load(url))
    model_reload.eval()
    return model_reload

def image_transfroms():# 把数据转换成模型指定大小，并转换成tensor类型
    return transforms.Compose([
        transforms.Scale(224),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
rows=[]
with open(csvfile_root,'r') as f:
        #image_id=image_id+image
        csv_file=csv.reader(f)
        rowss=[row for row in csv_file]
        rows=rowss

def get_result():#从文件夹里读取每一张图片并送入训练好的模型
    big_classes=os.listdir(infile)
   
    for big_class in big_classes:
        images=os.listdir(infile+big_class)
        i=big_class.find('_')
        class_name=big_class[:i]
        model=load_model(class_name)
        for image in images:
            img=Image.open(infile+big_class+'/'+image)
            img_tensor=image_transfroms()(img)
            img_tensor.resize_(1,3,224,224)
            strfeature=get_strfeature(model,img_tensor)
            image_id2=image_id+big_class+'/'+image
            for i,row in enumerate(rows):
                if row[0]==image_id2:
                    rows[i][2]=strfeature
                    break          
    return rows                    
rows=get_result()
time_elapsed=time.time()-start
print('读出和替换花费时间为{:.4f}分：{:.4f}秒'.format(time_elapsed//60,time_elapsed%60))
print('开始整体写入')
with open(csvfile_root,"w",newline='') as csvfile:
                        writer = csv.writer(csvfile)
                        for row in rows:
                            writer.writerow(row)
time_elapsed1=time.time()-start
print('end')
print('共花费时间为{:.4f}分：{:.4f}秒'.format(time_elapsed1//60,time_elapsed1%60))

start
读出和替换花费时间为20.0000分：20.9608秒
开始整体写入
end
共花费时间为20.0000分：21.0939秒
